## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Georgetown,MY,5.4112,100.3354,89.60,55,20,12.66,few clouds
1,Touba,CI,8.2833,-7.6833,74.91,80,100,7.67,overcast clouds
2,Mataura,NZ,-46.1927,168.8643,55.99,80,0,1.01,clear sky
3,Saint-Philippe,RE,-21.3585,55.7679,84.20,65,20,19.57,few clouds
4,Pochutla,MX,15.7432,-96.4661,70.00,83,0,2.89,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Georgetown,MY,5.4112,100.3354,89.60,55,20,12.66,few clouds
1,Touba,CI,8.2833,-7.6833,74.91,80,100,7.67,overcast clouds
3,Saint-Philippe,RE,-21.3585,55.7679,84.20,65,20,19.57,few clouds
4,Pochutla,MX,15.7432,-96.4661,70.00,83,0,2.89,clear sky
5,Dongzhen,CN,22.3797,110.9389,89.01,66,83,12.06,broken clouds
6,Vila Velha,BR,-20.3297,-40.2925,78.01,94,0,1.14,clear sky
7,Bredasdorp,ZA,-34.5322,20.0403,75.20,57,20,5.75,few clouds
9,Esperance,AU,-33.8667,121.9000,89.60,22,0,17.27,clear sky
13,Faanui,PF,-16.4833,-151.7500,82.36,73,0,9.28,clear sky
16,Port Elizabeth,ZA,-33.9180,25.5701,73.40,83,75,10.36,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   290
Country                289
Lat                    290
Lng                    290
Max Temp               290
Humidity               290
Cloudiness             290
Wind Speed             290
Current Description    290
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

/Users/giangnguyen/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


City                   289
Country                289
Lat                    289
Lng                    289
Max Temp               289
Humidity               289
Cloudiness             289
Wind Speed             289
Current Description    289
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,89.60,few clouds,5.4112,100.3354,
1,Touba,CI,74.91,overcast clouds,8.2833,-7.6833,
3,Saint-Philippe,RE,84.20,few clouds,-21.3585,55.7679,
4,Pochutla,MX,70.00,clear sky,15.7432,-96.4661,
5,Dongzhen,CN,89.01,broken clouds,22.3797,110.9389,
6,Vila Velha,BR,78.01,clear sky,-20.3297,-40.2925,
7,Bredasdorp,ZA,75.20,few clouds,-34.5322,20.0403,
9,Esperance,AU,89.60,clear sky,-33.8667,121.9000,
13,Faanui,PF,82.36,clear sky,-16.4833,-151.7500,
16,Port Elizabeth,ZA,73.40,broken clouds,-33.9180,25.5701,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))